In [ ]:
upstream = ['01-uniform-files']
product = None
input = None

# Features

In [1]:
import subprocess
import json
import pandas as pd
import re
import networkx as nx
import io
import numpy as np
import multiprocessing as mp
import json

In [3]:
reference = pd.read_csv(upstream['01-uniform-files']['data'], index_col=None)

In [5]:

def chain_sasa(x):
    buried = subprocess.run(f"/usr/local/bin/freesasa --format=json --depth=chain {x}", capture_output=True, shell=True)
    free = subprocess.run(f"/usr/local/bin/freesasa --format=json --separate-chains {x}", capture_output=True, shell=True)
    buried = json.loads(buried.stdout.decode())
    free = json.loads(free.stdout.decode())
    buried = pd.DataFrame.from_records([dict(label=x['label'], nres=x['n-residues'], **x['area']) for x in buried['results'][0]['structure'][0]['chains']])
    free = pd.DataFrame.from_records([dict(label=x['label'], nres=x['n-residues'], **x['area']) for x in map(lambda x: x['structure'][0]['chains'][0], free['results'])])
    all = pd.merge(buried, free, on=['label', 'nres'], suffixes=['_buried', '_free'])
    all['total_delta'] = all['total_buried'] - all['total_free']
    all['polar_delta'] = all['polar_buried'] - all['polar_free']
    all['apolar_delta'] = all['apolar_buried'] - all['apolar_free']
    return all.to_dict(orient='records')

In [6]:

def complex_sasa(x):

    buried = subprocess.run(f"/usr/local/bin/freesasa --format=json --depth=structure {x}", capture_output=True, shell=True)
    buried = json.loads(buried.stdout.decode())

    total_area =  buried['results'][0]['structure'][0]['area']['total']
    polar_area =  buried['results'][0]['structure'][0]['area']['polar']
    apolar_area = buried['results'][0]['structure'][0]['area']['apolar']

    free = subprocess.run(f"/usr/local/bin/freesasa --format=json --separate-chains {x}", capture_output=True, shell=True)
    free = json.loads(free.stdout.decode())
    free = pd.DataFrame.from_records([dict(label=x['label'], nres=x['n-residues'], **x['area']) for x in map(lambda x: x['structure'][0]['chains'][0], free['results'])])
    
    total_free_area = free['total'].sum()
    polar_free_area = free['polar'].sum()
    apolar_free_area = free['apolar'].sum()

    out = dict(
        total_area=total_area,
        buried_area=total_free_area - total_area,
        polar_area=polar_area,
        buried_polar_area=polar_free_area - polar_area,
        apolar_area=apolar_area,
        buried_apolar_area=apolar_free_area - apolar_area
    )

    return out

## Prodigy

In [7]:
x = 'nsdb-000001.clean.pdb'
u = ['A', 'B']

def prodigy_chain_interactions(x, u):
    out = subprocess.run(f"/home/bcz/bin/prodigy {x} --selection {u[0]} {u[1]}", capture_output=True, shell=True)
    y = out.stdout.decode()

    return dict(
        id = x.replace('.clean.pdb', ''),
        interface = '_'.join(sorted(u)),
        intermolecular_contacts  = int(re.search('No. of intermolecular contacts: (\d*)\n', y)[1]),
        charged_charged_contacts = int(re.search('No. of charged-charged contacts: (\d*)\n', y)[1]),
        charged_polar_contacts   = int(re.search('No. of charged-polar contacts: (\d*)\n', y)[1]),
        charged_apolar_contacts  = int(re.search('No. of charged-apolar contacts: (\d*)\n', y)[1]),
        apolar_polar_contacts    = int(re.search('No. of apolar-polar contacts: (\d*)\n', y)[1]),
        apolar_apolar_contacts   = int(re.search('No. of apolar-apolar contacts: (\d*)\n', y)[1]),
        apolar_nis_residues      = float(re.search('Percentage of apolar NIS residues: (\d*\.\d*)\n', y)[1]),
        charged_nis_residues     = float(re.search('Percentage of charged NIS residues: (\d*\.\d*)\n', y)[1]),
        binding_affinity         = float(re.search('Predicted binding affinity \(kcal.mol-1\):\s*(\-?\d*\.\d*)\n', y)[1])
    )

def prodigy_complex_stability(x):
    out = subprocess.run(f"/home/bcz/bin/prodigy {x}", capture_output=True, shell=True)
    y = out.stdout.decode()

    return dict(
        intermolecular_contacts  = int(re.search('No. of intermolecular contacts: (\d*)\n', y)[1]),
        charged_charged_contacts = int(re.search('No. of charged-charged contacts: (\d*)\n', y)[1]),
        charged_polar_contacts   = int(re.search('No. of charged-polar contacts: (\d*)\n', y)[1]),
        charged_apolar_contacts  = int(re.search('No. of charged-apolar contacts: (\d*)\n', y)[1]),
        apolar_polar_contacts    = int(re.search('No. of apolar-polar contacts: (\d*)\n', y)[1]),
        apolar_apolar_contacts   = int(re.search('No. of apolar-apolar contacts: (\d*)\n', y)[1]),
        apolar_nis_residues      = float(re.search('Percentage of apolar NIS residues: (\d*\.\d*)\n', y)[1]),
        charged_nis_residues     = float(re.search('Percentage of charged NIS residues: (\d*\.\d*)\n', y)[1]),
        binding_affinity         = float(re.search('Predicted binding affinity \(kcal.mol-1\):\s*(\-?\d*\.\d*)\n', y)[1])
    )


## Normal modes

In [8]:
import prody as pdy 

In [9]:
def generate_eigvals(file):
    x = pdy.parsePDB(file).select('name CA')
    g = pdy.GNM()
    g.buildKirchhoff(x)
    g.calcModes()
    y = g.getEigvals()
    rad = pdy.calcGyradius(x)
    return dict(
        eig0=y[0], eig1=y[1], eig2=y[2], eig3=y[3], eig4=y[4], rad=rad
    )

## Network

In [10]:
def obtain_network(file):
    ring = subprocess.run(f'/home/bcz/dev/ring-3.0.0/ring/bin/ring --best_edge -i  {file}',  capture_output=True, shell=True)
    ring = ring.stdout.decode().splitlines()

    header_edges_linenumber = ring.index('NodeId1\tInteraction\tNodeId2\tDistance\tAngle\tEnergy\tAtom1\tAtom2\tDonor\tPositive\tCation\tOrientation\tModel')
    header_nodes_linenumber = ring.index('NodeId\tChain\tPosition\tResidue\tDssp\tDegree\tBfactor_CA\tx\ty\tz\tpdbFileName\tModel')

    edges = pd.read_csv(io.StringIO('\n'.join(ring[header_edges_linenumber:header_nodes_linenumber])), sep='\t')
    nodes = pd.read_csv(io.StringIO('\n'.join(ring[header_nodes_linenumber:])), sep='\t')

    nodes = nodes.rename(columns=dict(Chain='chain', Position='resid', Residue='resname'))
    edges['Interaction'] = edges['Interaction'].apply(lambda x: x.split(':')[0])
    nodes.NodeId = nodes.NodeId.apply(lambda x : x.replace(':_:', ':'))

    g = nx.Graph()
    nodes.query('Degree > 0').apply(lambda x: g.add_node(x.NodeId.replace(':_:', ':'), residue=x.resname, resid=x.resid, chain=x.chain), axis=1)
    edges.apply(lambda x: g.add_edge(x.NodeId1.replace(':_:', ':'), x.NodeId2.replace(':_:', ':'), type=x.Interaction, energy=x.Energy), axis=1)
    nodes2remove = pd.DataFrame.from_records(list(g.degree()), columns=['id', 'degree']).query('degree < 1').id.to_list()
    g.remove_nodes_from(nodes2remove)
    g = g.subgraph(list(sorted(nx.connected_components(g), key=len, reverse=True))[0])
    return dict(
        avg_shortest_path=nx.average_shortest_path_length(g),
        avg_clustering=nx.average_clustering(g),
        avg_density=nx.density(g),
        avg_degree=np.mean(list(dict(g.degree()).values()))
    )
    

In [18]:
def featurize(x):
    path = 'pdb/' + x + '.clean.pdb'
    sasa = complex_sasa(path)
    prodigy = prodigy_complex_stability(path)
    eigvals = generate_eigvals(path)
    network = obtain_network(path)
    out =  dict(
        id=x, **sasa
    )
    out.update(prodigy)
    out.update(eigvals)
    out.update(network)
    return out

In [25]:
targets = reference['id'].to_list()

In [29]:
pool = mp.Pool(8)

results = [
    pool.apply_async(
        featurize, args=(x,)
    ) for x in targets
]
results = [p.get() for p in results]

In [ ]:
with open(product['data'], 'w') as f:
    json.dump(results, f, indent=4)